 # Pokemon Image Clustering #

In [38]:
# for loading/processing the images
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input
# for other things
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

In [39]:
import os

pokemons = []
rootdir = '../data/PokemonData'
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        pokemons.append([file, subdir.split('\\')[-1]])
        
print(pokemons[:10])

[['0282b2f3a22745f1a436054ea15a0ae5.jpg', 'Abra'], ['06b9eec4827d4d49b1b4c284308708df.jpg', 'Abra'], ['10a9f06ec6524c66b779ea80354f8519.jpg', 'Abra'], ['1788abb8b51f48509cfac8067bd99e14.jpg', 'Abra'], ['28cfad92ad934d1f9b579cbff4b5d012.jpg', 'Abra'], ['2eb2a528f9a247358452b3c740df69a0.jpg', 'Abra'], ['2fd28e699b7c4208acd1637fbad5df2d.jpeg', 'Abra'], ['32240b108a8140f8b31c495166fc453c.jpg', 'Abra'], ['34532bb006714727ade4075f0a72b92d.jpg', 'Abra'], ['3680c3f65a484c3ba05a7cb93e1d7ae3.jpg', 'Abra']]


In [40]:
pokemon_labels = pd.DataFrame(pokemons, columns = ['FileName', 'Label'])
# credit to https://towardsdatascience.com/how-to-cluster-images-based-on-visual-similarity-cd6e7209fe34
#    for help with image processing
# load the image as a 224x224 array
import matplotlib.image as mpimg
import imghdr
array_list = []
for pokemon in range(len(pokemon_labels)):
    img_path = '../data/PokemonData/'+pokemon_labels.iloc[pokemon]['Label']+'/'+pokemon_labels.iloc[pokemon]['FileName']
    img_type = imghdr.what(img_path)
    if img_type != "png" and img_type != "jpg" and img_type != "jpeg":
        os.remove(img_path)
    else:
        img = load_img(img_path, target_size=(224,224))
        # convert from 'PIL.Image.Image' to numpy array
        img = np.array(img)
        reshaped_img = img.reshape(224,224,3)
        x = preprocess_input(reshaped_img)
        array_list.append(x)

In [41]:
array_list

[array([[[ 36.060997 ,   2.2210007, 127.32     ],
         [ 36.060997 ,   2.2210007, 127.32     ],
         [ 39.060997 ,   5.2210007, 125.32     ],
         ...,
         [ 98.061    ,  16.221    , 124.32     ],
         [ 98.061    ,  16.221    , 124.32     ],
         [ 97.061    ,  15.221001 , 123.32     ]],
 
        [[ 34.060997 ,   3.2210007, 127.32     ],
         [ 35.060997 ,   4.2210007, 126.32     ],
         [ 37.060997 ,   5.2210007, 125.32     ],
         ...,
         [103.061    ,  21.221    , 129.32     ],
         [103.061    ,  21.221    , 129.32     ],
         [102.061    ,  20.221    , 128.32     ]],
 
        [[ 33.060997 ,   4.2210007, 128.32     ],
         [ 33.060997 ,   5.2210007, 126.32     ],
         [ 36.060997 ,   6.2210007, 126.32     ],
         ...,
         [100.061    ,  22.221    , 131.32     ],
         [ 99.061    ,  21.221    , 130.32     ],
         [ 99.061    ,  21.221    , 130.32     ]],
 
        ...,
 
        [[ 93.061    ,  50.221    